In [1]:
from modules import adquire, cleaning, create_df_checking, outputs
from numpy.random import choice
from time import sleep
import pygame
pygame.mixer.init()

pygame 2.0.0 (SDL 2.0.12, python 3.9.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [12]:
#drumroll = pygame.mixer.Sound('drumroll.wav')
#drumroll.play()

def main(): 
    df_students, df_katas = adquire()    
    df_students, df_katas = cleaning(df_students, df_katas)
    df,hora= create_df_checking(df_students, df_katas)    
    outputs(df,False)
    return df


if __name__ == '__main__':
    df = main()

AnaAGG presentará la kata keep-hydrated-1


slug,sum-mixed-array,keep-hydrated-1
username,,
amorenorp,True,True
alfredozavala14,True,True
anadeluna,True,True
AnaAGG,True,True
angelanavarro,True,True
antoniogarciagiron,True,True
xarlomi,False,True
CristinaCallejo,False,True
Florengutierrez,False,True
